In [3]:
# Dashboard Implementation: Interactive Fission Chain Reaction Visualization (Grid Layout & Improved Labels)

from IPython.core.display import HTML
HTML("""
<style>
.container { width: 100% !important; }
.output_wrapper, .output {
    max-width: 100% !important;
    flex: 1 1 auto;
}
</style>
""")


import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Output, GridspecLayout
from IPython.display import display, clear_output

# --- Calculation Functions ---

def neutron_population(k, generations):
    neutrons = [1]
    for _ in range(generations):
        neutrons.append(neutrons[-1] * k)
    return neutrons

def neutron_flux_distribution(geometry, size, enrichment):
    np.random.seed(0)
    flux = np.random.rand(size, size) * enrichment
    if geometry == "Cylinder":
        mask = np.sqrt((np.indices((size, size)) - size//2)**2).sum(0) < size//2
        flux *= mask
    elif geometry == "Sphere":
        Y, X = np.ogrid[:size, :size]
        mask = (X - size//2)**2 + (Y - size//2)**2 <= (size//2)**2
        flux *= mask
    return flux

def neutron_lifetime_distribution(moderation_rate, fuel_type):
    np.random.seed(1)
    mean_lifetime = {'U-235': 10, 'Pu-239': 8}
    lifetimes = np.random.exponential(mean_lifetime[fuel_type] / moderation_rate, 1000)
    return lifetimes

def energy_release(k, fuel_type, generations):
    base_energy = {'U-235': 200, 'Pu-239': 210} # MeV per fission
    neutron_counts = neutron_population(k, generations)
    return np.array(neutron_counts) * base_energy[fuel_type]

# --- Output Widgets ---
out_pop, out_flux, out_lifetime, out_energy = Output(), Output(), Output(), Output()

# --- Update Function ---
def update_plots(k, generations, geometry, enrichment, moderation_rate, fuel_type):
    size = 50

    # Neutron Population Growth
    with out_pop:
        clear_output(wait=True)
        neutrons = neutron_population(k, generations)
        plt.figure(figsize=(6,4))
        plt.plot(neutrons, marker='o')
        #plt.yscale('log')
        plt.title("Neutron Population Growth")
        plt.xlabel("Generation")
        plt.ylabel("Neutron Count")
        plt.grid(True)
        plt.show()

    # Average Neutron Flux by X-Axis
    with out_flux:
        clear_output(wait=True)
        flux = neutron_flux_distribution(geometry, size, enrichment)
        avg_flux_x = flux.mean(axis=0)  # Average over y-axis

        plt.figure(figsize=(6,4))
        plt.plot(avg_flux_x, color='crimson')
        plt.title("Average Neutron Flux Across Core (x-axis)")
        plt.xlabel("Core Position (x-axis)")
        plt.ylabel("Average Flux Intensity")
        plt.grid(True)
        plt.show()


    # Neutron Lifetime Distribution
    with out_lifetime:
        clear_output(wait=True)
        lifetimes = neutron_lifetime_distribution(moderation_rate, fuel_type)
        plt.figure(figsize=(6,4))
        plt.hist(lifetimes, bins=30, density=True, alpha=0.7)
        plt.title("Neutron Lifetime Distribution")
        plt.xlabel("Lifetime (μs)")
        plt.ylabel("Probability Density")
        plt.grid(True)
        plt.show()

    # Energy Release per Generation
    with out_energy:
        clear_output(wait=True)
        energy = energy_release(k, fuel_type, generations)
        plt.figure(figsize=(6,4))
        plt.bar(range(generations+1), energy, alpha=0.7)
        plt.yscale('log')
        plt.title("Energy Release per Generation")
        plt.xlabel("Generation")
        plt.ylabel("Energy (MeV)")
        plt.grid(True)
        plt.show()

# --- Widgets ---
k_slider = widgets.FloatSlider(min=0.5, max=2.0, step=0.01, value=1.0, description='k-value')
gen_slider = widgets.IntSlider(min=5, max=50, step=1, value=10, description='Generations')
geom_dropdown = widgets.Dropdown(options=['Slab', 'Cylinder', 'Sphere'], description='Geometry')
enrich_slider = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=1.0, description='Enrichment')
mod_slider = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=1.0, description='Moderation')
fuel_dropdown = widgets.Dropdown(options=['U-235', 'Pu-239'], description='Fuel Type')

ui = VBox([
    HBox([k_slider, gen_slider]),
    HBox([geom_dropdown, enrich_slider]),
    HBox([mod_slider, fuel_dropdown])
])

# Link Widgets
widgets.interactive_output(update_plots, {
    'k': k_slider, 'generations': gen_slider, 'geometry': geom_dropdown,
    'enrichment': enrich_slider, 'moderation_rate': mod_slider, 'fuel_type': fuel_dropdown
})

# Display plots in grid
plot_grid = GridspecLayout(2, 2, height='auto')
plot_grid[0, 0] = out_pop
plot_grid[0, 1] = out_flux
plot_grid[1, 0] = out_lifetime
plot_grid[1, 1] = out_energy

display(ui, plot_grid)


GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…